<a href="https://colab.research.google.com/github/Rikipratama/Rikipratama/blob/main/projek_gabung_file_excel_multiple_sheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas numpy openpyxl

In [16]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils import get_column_letter

def check_merged_cells(workbook, sheet_name):
    """Memeriksa dan menghapus baris yang berisi sel yang di-merge"""
    worksheet = workbook[sheet_name]
    merged_ranges = worksheet.merged_cells.ranges
    rows_to_remove = set()

    for merged_range in merged_ranges:
        # Mendapatkan baris yang terlibat dalam merge
        start_row = merged_range.min_row
        end_row = merged_range.max_row
        for row in range(start_row, end_row + 1):
            rows_to_remove.add(row)

    # Mengurutkan baris yang akan dihapus secara terbalik untuk menghindari masalah indeks
    return sorted(rows_to_remove, reverse=True)

def process_excel_file(input_file, output_file, exclude_sheets=None):
    """
    Memproses file Excel dengan menggabungkan multiple sheet menjadi satu
    dengan pengecualian sheet yang ditentukan.
    """
    if exclude_sheets is None:
        exclude_sheets = []

    # Membaca workbook dengan openpyxl untuk memeriksa merged cells
    workbook = openpyxl.load_workbook(input_file)

    # Menyiapkan list untuk menyimpan semua data yang sudah diproses
    all_data = []

    # Statistik status
    total_sheets = len(workbook.sheetnames)
    processed_sheets = 0
    skipped_sheets = 0
    skipped_sheets_list = []
    empty_sheets = 0
    empty_sheets_list = []
    excluded_sheets = 0

    # Memproses setiap sheet
    for sheet_name in workbook.sheetnames:
        if sheet_name in exclude_sheets:
            print(f"Sheet '{sheet_name}' dilewati sesuai permintaan.")
            excluded_sheets += 1
            continue

        print(f"Memproses sheet: {sheet_name}")

        # Cek apakah sheet kosong
        worksheet = workbook[sheet_name]
        if worksheet.max_row <= 1:  # Hanya memiliki header atau kosong
            print(f"Sheet '{sheet_name}' kosong. Dilewati.")
            empty_sheets += 1
            empty_sheets_list.append(sheet_name)
            continue

        # Membaca data dengan pandas menggunakan engine openpyxl
        try:
            df = pd.read_excel(input_file, sheet_name=sheet_name, engine='openpyxl')

            # Cek apakah dataframe kosong
            if df.empty:
                print(f"Sheet '{sheet_name}' kosong. Dilewati.")
                empty_sheets += 1
                empty_sheets_list.append(sheet_name)
                continue

            # Jika tidak ada kolom yang diperlukan, lanjut ke sheet berikutnya
            required_columns = ['PLU', 'BARCODE', 'Nama Barang', 'Satuan']
            if not all(col in df.columns for col in required_columns):
                print(f"Sheet '{sheet_name}' tidak memiliki semua kolom yang diperlukan. Dilewati.")
                skipped_sheets += 1
                skipped_sheets_list.append(sheet_name)
                continue
        except Exception as e:
            print(f"Error saat membaca sheet '{sheet_name}': {e}. Dilewati.")
            skipped_sheets += 1
            skipped_sheets_list.append(sheet_name)
            continue

        # Memeriksa merged cells dan mendapatkan baris yang harus dihapus
        rows_to_remove = check_merged_cells(workbook, sheet_name)

        # Menghapus baris yang memiliki merged cells
        if rows_to_remove:
            # Menyesuaikan indeks baris (pandas dimulai dari 0, openpyxl dari 1)
            rows_to_remove_adjusted = [r-1 for r in rows_to_remove]
            print(f"Menghapus {len(rows_to_remove)} baris dengan merged cells dari sheet '{sheet_name}'")
            df = df.drop(rows_to_remove_adjusted, errors='ignore')
            df = df.reset_index(drop=True)

        # Menambahkan kolom informasi sumber sheet
        df['SumberSheet'] = sheet_name

        # Pastikan PLU dan BARCODE adalah tipe data text
        df['PLU'] = df['PLU'].astype(str)
        df['BARCODE'] = df['BARCODE'].astype(str)

        # Pastikan Satuan adalah bilangan bulat
        try:
            df['Satuan'] = pd.to_numeric(df['Satuan'], errors='coerce').fillna(0).astype(int)
        except Exception as e:
            print(f"Error mengkonversi kolom Satuan ke integer di sheet '{sheet_name}': {e}")
            # Jika konversi gagal, tetap pertahankan kolom asli

        # Mengisi nilai yang kosong dengan "No Data"
        df = df.fillna("No Data")

        # Menyimpan data yang sudah diproses
        all_data.append(df)
        processed_sheets += 1

    # Menggabungkan semua data jika ada
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)

        # Pastikan hanya kolom yang diinginkan yang disimpan dengan urutan yang benar
        desired_columns = ['PLU', 'BARCODE', 'Nama Barang', 'Satuan', 'SumberSheet']
        combined_df = combined_df[desired_columns]

        # Menyimpan hasil ke file baru dengan engine openpyxl
        combined_df.to_excel(output_file, index=False, sheet_name='GabunganData', engine='openpyxl')
        print(f"\nFile berhasil disimpan: {output_file}")

        # Informasi status data
        print("\n===== LAPORAN STATUS PENGGABUNGAN =====")
        print(f"Total sheet dalam file: {total_sheets}")
        print(f"Sheet yang berhasil diproses: {processed_sheets}")
        print(f"Sheet yang dilewati karena dikecualikan: {excluded_sheets}")
        print(f"Sheet yang dilewati karena kosong: {empty_sheets}")
        if empty_sheets > 0:
            print(f"  List sheet kosong: {', '.join(empty_sheets_list)}")
        print(f"Sheet yang dilewati karena format tidak sesuai: {skipped_sheets}")
        if skipped_sheets > 0:
            print(f"  List sheet tidak sesuai: {', '.join(skipped_sheets_list)}")

        print("\n===== STATISTIK DATA HASIL GABUNGAN =====")
        print(f"Total baris data: {len(combined_df)}")
        for col in desired_columns:
            null_count = combined_df[col].isin(["No Data"]).sum()
            print(f"Kolom '{col}': {null_count} data kosong")

        # Statistik per sheet
        print("\n===== STATISTIK PER SHEET =====")
        sheet_stats = combined_df.groupby('SumberSheet').size()
        for sheet_name, count in sheet_stats.items():
            print(f"{sheet_name}: {count} baris")
    else:
        print("\n===== LAPORAN STATUS PENGGABUNGAN =====")
        print(f"Total sheet dalam file: {total_sheets}")
        print(f"Sheet yang berhasil diproses: {processed_sheets}")
        print(f"Sheet yang dilewati karena dikecualikan: {excluded_sheets}")
        print(f"Sheet yang dilewati karena kosong: {empty_sheets}")
        if empty_sheets > 0:
            print(f"  List sheet kosong: {', '.join(empty_sheets_list)}")
        print(f"Sheet yang dilewati karena format tidak sesuai: {skipped_sheets}")
        if skipped_sheets > 0:
            print(f"  List sheet tidak sesuai: {', '.join(skipped_sheets_list)}")
        print("\nTidak ada data yang diproses. Pastikan file Excel memiliki sheet dengan format yang sesuai.")

if __name__ == "__main__":
    # Contoh penggunaan:
    input_file = input("Masukkan path file Excel yang akan diproses: ")
    output_file = input("Masukkan path file output: ")

    # Pastikan file output memiliki ekstensi .xlsx
    if not output_file.lower().endswith('.xlsx'):
        output_file += '.xlsx'

    # Tanyakan sheet yang akan dikecualikan
    exclude_input = input("Masukkan nama sheet yang tidak ingin digabungkan (pisahkan dengan koma jika lebih dari satu): ")
    exclude_sheets = [s.strip() for s in exclude_input.split(',')] if exclude_input else []

    try:
        # Proses file
        process_excel_file(input_file, output_file, exclude_sheets)
    except Exception as e:
        print(f"Terjadi error: {e}")
        print("Pastikan file yang diinput ada dan memiliki format Excel (.xls atau .xlsx)")

Masukkan path file Excel yang akan diproses: /content/SO 11 MARET 2025 FIXX (2) - Copy.xlsx
Masukkan path file output: gabungan data SO Team Pak Adri
Masukkan nama sheet yang tidak ingin digabungkan (pisahkan dengan koma jika lebih dari satu): MASTER GT, Sheet1, MASTER
Sheet 'MASTER GT' dilewati sesuai permintaan.
Sheet 'Sheet1' dilewati sesuai permintaan.
Sheet 'MASTER' dilewati sesuai permintaan.
Memproses sheet: WALLS
Memproses sheet: FLOOR MINUMAN
Memproses sheet: RAK F5
Memproses sheet: CAMPURAN
Memproses sheet: RAK 001
Memproses sheet: RAK 002
Memproses sheet: RAK 003
Menghapus 86 baris dengan merged cells dari sheet 'RAK 003'
Memproses sheet: RAK 004
Memproses sheet: RAK 005
Memproses sheet: RAK 006
Memproses sheet: RAK 007
Memproses sheet: RAK 008
Memproses sheet: RAK 009
Memproses sheet: RAK 010
Memproses sheet: RAK 011
Memproses sheet: RAK 012
Memproses sheet: RAK 013
Memproses sheet: RAK 014
Memproses sheet: RAK 015
Memproses sheet: RAK 016
Memproses sheet: RAK 17
Memproses 